In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [7]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000018AFFBE70D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018AFF251550>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="who are you")]).content

"I am Gemma, an open-weights AI assistant. I'm a large language model trained by Google DeepMind.\n\nHere are some key things to know about me:\n\n* **Open-weights:** My weights are publicly accessible, meaning anyone can see and use the underlying code that makes me work. This promotes transparency and collaboration in the AI community.\n* **Text-only:** I can only communicate through text. I can't generate images, sound, or videos.\n* **Knowledge cutoff:** My knowledge is based on the data I was trained on, which has a cutoff point. I don't have access to real-time information or the ability to search the internet.\n* **Created by the Gemma team:** I was developed by a team of engineers and researchers at Google DeepMind.\n\nI'm here to help you with a variety of tasks, such as:\n\n* **Generating creative content:**\n\nI can write stories, poems, and even code.\n* **Answering your questions:** I can provide information on a wide range of topics, based on my training data.\n* **Summar

In [17]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="hi , my name is rudra I am AI Engineer"),
        AIMessage(content="hello Rudra It's nice to meet you."),
        HumanMessage(content="hey what's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Rudra and that you are an AI Engineer!  😄  \n\nIs there anything specific you'd like to talk about related to AI engineering? I'm happy to chat about it.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 53, 'total_tokens': 101, 'completion_time': 0.087272727, 'prompt_time': 0.003545578, 'queue_time': 0.248553501, 'total_time': 0.090818305}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--e523f062-a181-4fe8-9e80-fa9535db42b7-0', usage_metadata={'input_tokens': 53, 'output_tokens': 48, 'total_tokens': 101})

## Message History
we can use a Message History class to wrap our model and make it stateful.This will keep track of inputs and outputs of the model.

In [34]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [ ]:
config = {"configurable": {"session_id": "chat1"}}
with_message_history.invoke(
    [HumanMessage(content="HI hello I am Rudra")],
    config
) 

AIMessage(content="Hello Rudra! 👋 \n\nIt's nice to meet you. What can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 15, 'total_tokens': 44, 'completion_time': 0.052727273, 'prompt_time': 0.00194515, 'queue_time': 0.24726218, 'total_time': 0.054672423}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--6f54d030-33d8-47f7-8e71-9c1727fb2217-0', usage_metadata={'input_tokens': 15, 'output_tokens': 29, 'total_tokens': 44})

In [40]:
config1 ={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
)

In [45]:
config1 ={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
)

In [46]:
response

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 14, 'total_tokens': 56, 'completion_time': 0.076363636, 'prompt_time': 0.001879069, 'queue_time': 0.24518675999999998, 'total_time': 0.078242705}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--22eb80ba-08a4-4fd1-9835-00ff7027732e-0', usage_metadata={'input_tokens': 14, 'output_tokens': 42, 'total_tokens': 56})

## Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's not make that a bit more complicated.

In [52]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
pt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.answer all the question to the next of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=pt|model

In [53]:
chain.invoke({"messages":[HumanMessage(content="My name is Rudra")]})

AIMessage(content="Hello Rudra, it's nice to meet you!  \n\nI'm ready to answer your questions to the best of my ability. Just ask away! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 30, 'total_tokens': 69, 'completion_time': 0.070909091, 'prompt_time': 0.002088659, 'queue_time': 0.24554647100000002, 'total_time': 0.07299775}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--70739de9-8ad2-4ec4-80bd-e2a0a852aa8b-0', usage_metadata={'input_tokens': 30, 'output_tokens': 39, 'total_tokens': 69})

In [61]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

config={"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is rudra")],
    config=config
)

response.content

"Hi Rudra, it's nice to meet you! \n\nIt seems like you've introduced yourself a few times. What can I do for you?  \n\nI'm ready for any questions you have! 😊  \n\n"

In [62]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.answer all the question to the next of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [98]:
response = chain.invoke({"messages":[HumanMessage(content="hi my name is Rudra")],"language":"Gujrati"})

In [99]:
response.content

'નમસ્તે રુદ્રજી! \n\nમને ખુશી છે કે તમે મને "કેટલે છે" બોલવા આવ્યા છો. હું તમારી સहायક તરીકે તમારા કહેવાલા પ્રત્યે જવાબ આપવા માટે તૈયાર છું. \n\nતમારી સમજાવી શકાય તેવા પ્રશ્નો પૂછો, હું મારા શક્યતમ પ્રયત્નથી ઉત્તર આપવાનો પ્રયત્ન કરીશ. 😊\n\n'

In [86]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [93]:
config4 = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="what my name")], "language": "Hindi"},
    config=config4,
)
response.content

'आपका नाम Rudra है।  😊 \n'

## Managing the Conversation History
one important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM.  

'trim_messages' helper to reduce how many messages we're sending to the model.trimmer allows us to specify how many tokens we want to keep,

In [103]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens=70,
    strategy='last',
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages=[
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content='hi!'),
    HumanMessage(content="haing fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

c:\Users\Admin\Desktop\Gen_AI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='haing fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [106]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer) | prompt | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what ice cream do i like")],
        "language": "Hindi"
    }
)
response.content

'मुझे नहीं पता कि तुम्हें कौन सा आइसक्रीम पसंद है!  क्या तुम मुझे बता सकते हो कि तुम्हें किस स्वाद में आइसक्रीम पसंद है? 🍦  \n\n'

In [111]:
## let's wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key = 'messages',
)

config={"configurable":{"session_id":"chat5"}}

In [114]:
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  messages: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x0000018AFFC8B600>, input_messages_key='messages', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

## Vector stores and retrievers

These abstractions are designed to support retrieval of data-- from databased and other sources-- for integration with LLM workflows.

### Documents
Langchain implements a document abstraction,intended to represent a unit of text associated metadata.
Page_content: astring representing the content;
- metadata: a dict containing arbitraty metadata. The metadata attribute can capture information about the source of the document,
individual Document object often represents a chunk of large document.

In [16]:
from langchain_core.documents import Document
documents = [
    Document(
        page_content="Dogs are great companions, know for their loylty and freiendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
]

In [17]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loylty and freiendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [19]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()
groq_api = os.getenv('GROQ_API_KEY')

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
LLM = ChatGroq(groq_api_key=groq_api, model="Llama3-8b-8192")
LLM

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D9722E1250>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D972506B90>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model="Zybg/synthetic-clinical-embedding-model")

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

In [21]:
from langchain_chroma import Chroma

vectorstore=Chroma.from_documents(documents,embeddings)
vectorstore

In [8]:
## vectorStores and use Chroma
vectorstore.similarity_search("cat")

[Document(id='3ad4b594-528d-4293-af3f-715f064819c6', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='4a0addda-92d4-4339-9967-0788d474d02f', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='b2cc97a8-570b-4898-9a7e-d359519bf4f1', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loylty and freiendliness.'),
 Document(id='158c6156-f567-4d00-a1a0-cdbf7567d275', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loylty and freiendliness.')]

In [10]:
await vectorstore.asimilarity_search("cat")

[Document(id='3ad4b594-528d-4293-af3f-715f064819c6', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='4a0addda-92d4-4339-9967-0788d474d02f', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='b2cc97a8-570b-4898-9a7e-d359519bf4f1', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loylty and freiendliness.'),
 Document(id='158c6156-f567-4d00-a1a0-cdbf7567d275', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loylty and freiendliness.')]

## Retrievers
Langchain VectorStore Objects do not subclass Runnable, and so cannot immediately be integrated into Langchain Expression Language chains.

In [22]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])


[[Document(id='f1fcf2b0-0d97-4ca9-9b7e-44682907a6d4', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='158c6156-f567-4d00-a1a0-cdbf7567d275', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loylty and freiendliness.')]]

Vectorsotes implement an as_retriever method that will generate a Retriever ,specifically a VectorStoreRetriever. These Retrieves include specific search_type and search_kwargs attributes that identify

In [13]:
r = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1}
)
retriever.batch(['dog','elephant'])

[[Document(id='b2cc97a8-570b-4898-9a7e-d359519bf4f1', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loylty and freiendliness.')],
 [Document(id='b4992932-eae6-470f-b81b-5851d3f3f251', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message="""
    Answer this question using the provided context only.

    {question}

    context:
    {context}
"""
prompt = ChatPromptTemplate.from_messages([("human",message)])

rag_chain={"context":retriever,"question":RunnablePassthrough()} | prompt | LLM

response = rag_chain.invoke("tell me about Goldfish")
print(response.content)

According to the provided context, Goldfish are popular pets for beginners, requiring relatively simple care.


In [1]:
pip uninstall pydantic -y

Found existing installation: pydantic 1.10.22
Uninstalling pydantic-1.10.22:
  Successfully uninstalled pydantic-1.10.22
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.


In [2]:
pip install pydantic

  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
